In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageSequence
import glob
import os
from tqdm import tqdm

conversion = np.linspace(0.01,0.99,98)
temperature = np.linspace(470,600,98)

def P2(a):
    return 2*a**(1/2)

def P3(a):
    return 3*a**(2/3)

def P4(a):
    return 4*a**(3/4)

def A2(a):
    return 2*(1 - a)*(-np.log(1 - a))**(1/2)

def A3(a):
    return 3*(1 - a)*(-np.log(1 - a))**(2/3)

def A4(a):
    return 4*(1 - a)*(-np.log(1 - a))**(3/4)

def B1(a):
    return a*(1 - a)*np.log(a/(1 - a))

def R2(a):
    return 2*(1 - a)**(1/2)

def R3(a):
    return 3*(1 - a)**(2/3)

def D1(a):
    return 1/2*a

def D2(a):
    return -(np.log(1 - a))**(-1)

def D3(a):
    return 3*(1 - a)**(2/3)/(2*(1-(1-a)**(1/3)))

def D4(a):
    return 3/2*((1 - a)**(-1/3)-1)

def F0_R1(a):
    return 1

def F1(a):
    return (1 - a)

def F2(a):
    return (1 - a)**2

def F3(a):
    return (1 - a)**3

# Создаем словарь функций
model_funcs = {
    'Power law (P2)': P2,
    'Power law (P3)': P3,
    'Power law (P4)': P4,
    'Avarami-Erofe’ev (A2)': A2,
    'Avarami-Erofe’ev (A3)': A3,
    'Avarami-Erofe’ev (A4)': A4,
    'Prout–Tompkins (B1)': B1,
    'Contracting area (R2)': R2,
    'Contracting volume (R3)': R3,
    '1-D diffusion (D1)': D1,
    '2-D diffusion (D2)': D2,
    '3-D diffusion-Jander eqn.(D3)': D3,
    'Ginstling–Brounshtein (D4)': D4,
    'Zero-order (F0/R1)': F0_R1,
    'First-order (F1)': F1,
    'Second-order (F2)': F2,
    'Third-order (F3)': F3
}

# Проходим по каждой строке в DataFrame с моделями
for index, row in tqdm(models_df.iterrows(), total=models_df.shape[0]):
    # Извлекаем функцию модели из словаря функций
    model_func = model_funcs[row['Model']]

    images = []  # список для сохранения изображений графиков

    Ea_values = []  # список для хранения значений Ea
    A_values = []   # список для хранения значений A

    # Проходим по каждому значению Ea в диапазоне от 5000 до 150000 с шагом 500
    for Ea in tqdm(range(1000, 220001, 10000), desc=f"Processing model {row['Model']}"):
        try:
            # Вычисляем функцию, основанную на модели и преобразуем её с учетом Ea и температуры
            func = model_func(conversion[1:-1]) * np.exp(-Ea / (R * temperature[1:-1]))
            # Вычисляем коэффициент A как отношение последнего элемента конверсии к последнему элементу функции
            A = conversion[1:-1][-1] / func.max()
            # Получаем значение подстроенной модели
            model_fit = func*A           

            # Добавляем значения Ea и A в соответствующие списки
            Ea_values.append(Ea)
            A_values.append(A)
            
            fig, axs = plt.subplots(2)

            # Первый график: func от temperature[1:-1]
            axs[0].plot(temperature[1:-1], model_fit)            
            axs[0].set_title(row['Model'])
            axs[0].text(0.05, 0.95, f'ln(A): {np.round(np.log(A),2)}, Ea: {Ea}', ha='left', va='top', 
                        transform=axs[0].transAxes, fontsize=16)
            axs[0].tick_params(axis='both', which='major', labelsize=SMALL_SIZE)
            axs[0].set_xlabel('Температура, К', fontsize=MEDIUM_SIZE)
            axs[0].set_ylabel('Конверсия', fontsize=MEDIUM_SIZE)

            # Второй график: np.log(A) от Ea
            axs[1].plot(Ea_values, [np.log(a) for a in A_values], label='log(A) vs Ea')
            axs[1].tick_params(axis='both', which='major', labelsize=SMALL_SIZE)
            axs[1].set_xlabel('Ea', fontsize=MEDIUM_SIZE)
            axs[1].set_ylabel('log(A)', fontsize=MEDIUM_SIZE)

            # Сохраняем график как изображение в папке models_gifs
            fig_name = f"models_gifs/{row['Model']}_Ea_{Ea}.png"
            plt.savefig(fig_name)
            images.append(fig_name)

            plt.close(fig)  # закрываем фигуру, чтобы она не отображалась в notebook
            
        except Exception as e:
           # Если возникает ошибка, выводим информацию о ней и прерываем текущий цикл
            print(f"Error occurred with model: {row['Model']} and Ea: {Ea}. Error: {e}")
            break  # Если произошла ошибка, прерываем цикл для этой модели и переходим к следующей модели

    # Создаем gif анимацию
    gif_name = f"models_gifs/{row['Model']}.gif"
    frames = []
    for image in images:
        with Image.open(image) as img:
            frames.append(img.copy())
    if frames:
        frames[0].save(fp=gif_name, format='GIF', append_images=frames[1:],
                    save_all=True, duration=60, loop=0)

    # Удаляем временные изображения
    for image in images:
        os.remove(image)
